<img src="https://github.com/rjpost20/Onramp-Project/blob/main/data/pexels-vishnu-r-nair-1105666.jpg?raw=true">
Image by <a href="https://www.pexels.com/@vishnurnair/" >Vishnu R Nair</a> on <a href="https://www.pexels.com/photo/people-at-concert-1105666/" >Pexels.com</a>

# Onramp x Vanguard Spotify Project

## By Ryan Posternak

<br>

## Imports

In [3]:
import numpy as np
import pandas as pd

import http.client
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials